## Modules

In [1]:
import os
import sys
sys.path.append('..')

import numpy as np
import pandas as pd

from datetime import datetime, timedelta
from itertools import product
from pandas import ExcelWriter
from scipy.stats import norm

import plotly.graph_objects as go
from plotly.subplots import make_subplots

# `bsoption` module
from bsoption.bsmodel import *

## Single option pricing

### Sample ATM call (with given number of days)

In [2]:
stock1 = 200    # Underlying price
strike1 = 200   # strike price 
numday1 = 45   # number of days to expiry 
sig1 = 50   # IV ( 50% = 0.5)

BStest1 = BSModel(stock1, strike1, numday1, sig1/100)
print(f"Call Price: {BStest1.cprice}")
print(f"Call delta: {BStest1.cdelta}")
print(f"Call vega: {BStest1.vega}")
print(f"Call theta: {BStest1.ctheta}")

Call Price: 13.989829955085767
Call delta: 0.5349745748877144
Call vega: 0.27907868152712395
Call theta: -0.1550437119595133


### Sample OTM put (with expiry date & current date)

In [3]:
stock2 = 315   # Underlying price
strike2 = 300   # strike price 
tradedate2 = datetime(2021, 5, 10)
expiry2 = datetime(2021, 6, 18)
numday2 = (expiry2 - tradedate2).days    # number of days to expiry 
sig2 = 27   # IV 27%


BStest2 = BSModel(stock2, strike2, numday2, sig2/100)
print(f"Put Price: {BStest2.pprice}")
print(f"Put delta: {BStest2.pdelta}")
print(f"Put vega: {BStest2.vega}")
print(f"Put theta: {BStest2.ptheta}")

Put Price: 4.934954358341514
Put delta: -0.27527133493466915
Put vega: 0.34373753899635917
Put theta: -0.11898607119104739


### Payoff diagram: LONG-side, final price only

In [8]:
stock3 = 22000  # Underlying price
strike3 = 23000   # strike price 
tradedate3 = datetime(2022, 6, 6)
expiry3 = datetime(2022, 6, 30)
numday3 = (expiry3 - tradedate3).days    # number of days to expiry 
sig3 = 30   # IV 24%

BStest3 = BSModel(stock3, strike3, numday3, sig3/100)
BStest3.getpayoff(opside='LONG')

,spot,expC,expP
0,19550.0,0.0,3450.0
1,20240.0,0.0,2760.0
2,20930.0,0.0,2070.0
3,21620.0,0.0,1380.0
4,22310.0,0.0,690.0
5,23000.0,0.0,0.0
6,23690.0,690.0,0.0
7,24380.0,1380.0,0.0
8,25070.0,2070.0,0.0
9,25760.0,2760.0,0.0


### Payoff diagram: SHORT-side, final price only

In [9]:
BStest3.getpayoff(opside='SHORT')

,spot,expC,expP
0,19550.0,-0.0,-3450.0
1,20240.0,-0.0,-2760.0
2,20930.0,-0.0,-2070.0
3,21620.0,-0.0,-1380.0
4,22310.0,-0.0,-690.0
5,23000.0,-0.0,-0.0
6,23690.0,-690.0,-0.0
7,24380.0,-1380.0,-0.0
8,25070.0,-2070.0,-0.0
9,25760.0,-2760.0,-0.0


### Include some pre-expiry payoff curve

In [10]:
BStest3.getpayoff(preexpiry=True, opside='LONG')

,spot,expC,expP,7dayC,7dayP,14dayC,14dayP,28dayC,28dayP,56dayC,56dayP
0,19550.0,0.0,3450.0,0.01,3450.01,1.06,3451.06,16.81,3466.81,94.72,3544.72
1,20240.0,0.0,2760.0,0.26,2760.26,6.63,2766.63,48.07,2808.07,177.99,2937.99
2,20930.0,0.0,2070.0,3.64,2073.64,29.59,2099.59,116.61,2186.61,308.19,2378.19
3,21620.0,0.0,1380.0,27.81,1407.81,98.53,1478.53,245.16,1625.16,496.70,1876.70
4,22310.0,0.0,690.0,127.06,817.06,255.71,945.71,455.62,1145.62,751.85,1441.85
5,23000.0,0.0,0.0,381.18,381.18,539.03,539.03,762.20,762.20,1077.60,1077.60
6,23690.0,690.0,0.0,825.87,135.87,959.91,269.91,1167.04,477.04,1473.14,783.14
7,24380.0,1380.0,0.0,1415.87,35.87,1497.72,117.72,1660.39,280.39,1933.45,553.45
8,25070.0,2070.0,0.0,2076.92,6.92,2114.57,44.57,2224.72,154.72,2450.49,380.49
9,25760.0,2760.0,0.0,2760.98,0.98,2774.66,14.66,2840.21,80.21,3014.61,254.61


In [11]:
BStest3.getpayoff(preexpiry=True, opside='SHORT')

,spot,expC,expP,7dayC,7dayP,14dayC,14dayP,28dayC,28dayP,56dayC,56dayP
0,19550.0,-0.0,-3450.0,-0.01,-3450.01,-1.06,-3451.06,-16.81,-3466.81,-94.72,-3544.72
1,20240.0,-0.0,-2760.0,-0.26,-2760.26,-6.63,-2766.63,-48.07,-2808.07,-177.99,-2937.99
2,20930.0,-0.0,-2070.0,-3.64,-2073.64,-29.59,-2099.59,-116.61,-2186.61,-308.19,-2378.19
3,21620.0,-0.0,-1380.0,-27.81,-1407.81,-98.53,-1478.53,-245.16,-1625.16,-496.70,-1876.70
4,22310.0,-0.0,-690.0,-127.06,-817.06,-255.71,-945.71,-455.62,-1145.62,-751.85,-1441.85
5,23000.0,-0.0,-0.0,-381.18,-381.18,-539.03,-539.03,-762.20,-762.20,-1077.60,-1077.60
6,23690.0,-690.0,-0.0,-825.87,-135.87,-959.91,-269.91,-1167.04,-477.04,-1473.14,-783.14
7,24380.0,-1380.0,-0.0,-1415.87,-35.87,-1497.72,-117.72,-1660.39,-280.39,-1933.45,-553.45
8,25070.0,-2070.0,-0.0,-2076.92,-6.92,-2114.57,-44.57,-2224.72,-154.72,-2450.49,-380.49
9,25760.0,-2760.0,-0.0,-2760.98,-0.98,-2774.66,-14.66,-2840.21,-80.21,-3014.61,-254.61


## Finding IV by bisection method

### Estimate a single option IV

In [ ]:
stock4 = 3500  # Underlying price
strike4 = 4000 # Strike price
tradedate4 = datetime(2022, 4, 4)  # Current trading day
expiry4 = datetime(2022, 4, 29)  # Expiry date
numday4 = (expiry4 - tradedate4).days
opprice4 = 66
optype4 = 'C'

sig4 = getivbisect(stock4, strike4, numday4, opprice4, optype4)
print(f"IV: {sig4}%")

### Verify IV result by `BSModel()`

In [ ]:
BStest4 = BSModel(stock4, strike4, numday4, sig4/100)
print(f"Call Price: {BStest4.cprice}")
print(f"Call delta: {BStest4.cdelta}")
print(f"Call theta: {BStest4.ctheta}")
print(f"Put Price: {BStest4.pprice}")
print(f"Put delta: {BStest4.pdelta}")
print(f"Put theta: {BStest4.ptheta}")
print(f"Option vega: {BStest4.vega}")

### Verify Put-Call Parity (PCP)

In [ ]:
diffop4 = BStest4.pprice - BStest4.cprice
diffpr4 = strike4 - stock4
print(f'Difference of Put - call: {diffop4}')
print(f'Difference of Strike - Forward: {diffop4}')